# Notebook for Working with SMF Through its REST API — Basic Actions

This Jupyter Notebook is aimed at showing how you can use the REST interface in Oracle Solaris to work with the [Solaris Managment Facility](https://docs.oracle.com/cd/E37838_01/html/E60998/gmteb.html#scrolltoc) (SMF). The REST API is layered on top of the [Oracle Solaris Remote Administration Deamon (RAD)](https://docs.oracle.com/cd/E37838_01/html/E68270/gpzxz.html#scrolltoc), and gives access to all the RAD modules through REST. This notebook is using Python to run all the tasks. </br></br>
In this notebook we'll show how to connect to the REST interface and how to perform the various tasks you can do through REST.

> *Note 1* — You'll need to enable the `svc:/system/rad:remote` [service](https://docs.oracle.com/cd/E37838_01/html/E68270/gpzpd.html#scrolltoc) to be able to remotely connect. </br> 
> *Note 2* — If the server doesn't have a certification signed by a public CA then pull in `/etc/certs/localhost/host-ca/hostca.crt` from your server and point to its location in the JSON file below. </br> 
> *Note 3* — To do actions beyond looking at the services, like enabling or disabling a service, the user you use to connect to the system will need to have either the `Service Management` or `Service Operator` RBAC profiles set to be able perform these tasks. For more info [see `smf_security(7)`](https://docs.oracle.com/cd/E88353_01/html/E37853/smf-security-7.html#scrolltoc).</br>
> *Note 4* — This notebook was written with Python 3.7

Here are the steps on what to do.

## Introduction to the SMF API

The base URI of the Compliance API is: `api/com.oracle.solaris.rad.smf/1.0/`

The API has three main interfaces:

- **Master** — URI: `Master/` — This is a top-level object providing access to all services and instances on the system, this way you can find the services you're looking for.
- **Service** — URI: `Service/` — This object allows interaction with a specific SMF service, to get information about it, get its property groups, and properties, and edit them.
- **Instance** — URI: `Instance/` — This object represents an SMF instance, and allows interaction with the running instance.


The latter two interfaces, **Service** and **Instance**, are derived from a common **Entity** interface (URI: `Entity/`). They inherit most of their methods from **Entity**.  

Because there are many different things you can do through SMF, this notebook will focus how to use Master, Service, and Instance interfaces to do a basic lookup of a specific service, to discover its property groups, change one of its properties, and enable or disable the service. 

In this case we'll be looking at the `svc:/application/security/compliance:default` service which will run Compliance Assessments at a scheduled cadence.

In the second half of the notebook we'll be altering Property Values and turning a service instance on and off, make sure the user you're using to connect to the server has the `Service Management` or `Service Operator` RBAC profiles set to be able to run these steps.

---

## Imports and Setting Variables

First to import all the Python libraries:

In [1]:
import requests
import json
import base64
import os
import time, datetime
import pandas as pd

Turning off warnings thrown by Python:

In [2]:
import warnings
warnings.filterwarnings('ignore')

---
## Base Functions

Next we define the base functions used to make the connection with the REST interface. The first is to establish a session, the second is for the regular GET, PUT, POST, and DELETE methods.

The function to establish a session:

In [3]:
def rad_rest_login(session_name, server_connection_info, base_authentication_uri):
    login_url = 'https://{0}:{1}/{2}'.format(*list(map(server_connection_info.get, ['server_name', 'server_port'])), base_authentication_uri)
    print("Logging in with this URL: " + login_url)

    try:
        response = session_name.post(login_url, json = server_connection_info['config_json'], verify = server_connection_info['cert_location'])
    except:
        print('no connection')
        response = 'empty'
    
    return response

The function to run regular requests:

In [4]:
def rad_rest_request(request_type, session_name, server_connection_info, rad_rest_uri, payload = None):
    
    # Create a list of variables
    query_vars = list(map(server_connection_info.get, ['server_name', 'server_port']))
    query_vars.append(rad_rest_uri)

    # Create query url to be used
    query_url = 'https://{0}:{1}/{2}'.format(*query_vars)

    response = session_name.request(request_type, query_url, json = payload)
    
    return response

Two extra functions for convenience to convert Solaris timestamps to something human-readable, which might be useful later on:

In [5]:
def timestamp_from_solaris_time(value):
    return datetime.datetime.fromtimestamp(value/1000)

In [6]:
def str_from_solaris_time(value):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(value/1000))

---

## Load Remote System Connection Information

Now define the location of the remote system connection information:

In [7]:
config_filename = '../base_login_info.json'

The system specific information is located in a JSON file located in the same directory as the notebook, and is structured like this: </br>

```json
{
    "server_name": "<ip address>",
    "server_port": "<RAD Remote port>",
    "cert_location": "<path_to_cert>",
    "config_json": {
        "username": "<username>",
        "password": "<password>",
        "scheme": "pam",
        "preserve": true,
        "timeout": -1
    }
}
``` 

We advise to securely connect to the remote server over https however this will require the use of a certificate, in many cases the remote server will have a certificate signed by a public CA, if so please set `"cert_location": true`. If, for example in the case of a demo system, the certificate isn't signed download the `/etc/certs/localhost/host-ca/hostca.crt` file and refer to it's location with `"cert_location": "<path_to_cert>"`. If you don't want to use a certificate at all set `"cert_location": false` instead of the location of the cert file. Note this last option doesn't validate that you're connecting to correct server.

Loading this system specific information:

In [8]:
if config_filename:
    with open(config_filename, 'r') as f:
        server_connection_info = json.load(f)
        
server_connection_info['server_name']

't8ldom3.us.oracle.com'

---
---
## Connecting to the SMF Interface

We start with the example on how to connect to the server to get the list of services on the system and work with this. 

### Defining Interface URIs

Before we start we define the various URI variables:

In [9]:
base_authentication_uri = 'api/authentication/1.0/Session/'

base_smf_uri = 'api/com.oracle.solaris.rad.smf/1.0/'
smf_master_uri = '{}Master/'.format(base_smf_uri)
smf_instance_uri = '{}Instance/'.format(base_smf_uri)
smf_service_uri = '{}Service/'.format(base_smf_uri)
smf_entity_uri = '{}Entity/'.format(base_smf_uri)

### Logging in and setting up a session

First establish the session and bring in the necessary credentials. Note we're printing the results to give a better insight on what's coming back, we won't be doing this for every request just a few to give you a feel:

In [10]:
s = requests.Session()

login_answer = rad_rest_login(s, server_connection_info, base_authentication_uri)

# printing the answers coming back to show the proces.
print(login_answer.status_code)
print(login_answer.text)

Logging in with this URL: https://t8ldom3.us.oracle.com:6788/api/authentication/1.0/Session/
201
{
        "status": "success",
        "payload": {
                "href": "/api/com.oracle.solaris.rad.authentication/1.0/Session/_rad_reference/3328"
        }
}


---
### Getting the List of SMF Services

To get to the list of SMF services use the `instances` method added to the Master interface using the `GET` method:<br>
`https://{{server}}:6788/api/com.oracle.solaris.rad.smf/1.0/Master/instances`

This will return a list of dicts, one for each service containing basic information about the given service:

In [11]:
service_list = rad_rest_request('GET', s, server_connection_info, '{}/instances'.format(smf_master_uri))
service_list_text = json.loads(service_list.text)

print(service_list.status_code, '\n')
print(service_list_text)

200 

{'status': 'success', 'payload': [{'fmri': 'svc:/application/cups/in-lpd:default', 'objectName': 'api/com.oracle.solaris.rad.smf/1.0/Instance/application%2Fcups%2Fin-lpd,default', 'stime': 1601400264140, 'state': 'ONLINE'}, {'fmri': 'svc:/application/cups/scheduler:default', 'objectName': 'api/com.oracle.solaris.rad.smf/1.0/Instance/application%2Fcups%2Fscheduler,default', 'stime': 1601392118664, 'state': 'ONLINE'}, {'fmri': 'svc:/application/desktop-cache/desktop-mime-cache:default', 'objectName': 'api/com.oracle.solaris.rad.smf/1.0/Instance/application%2Fdesktop-cache%2Fdesktop-mime-cache,default', 'stime': 1601392633844, 'state': 'ONLINE'}, {'fmri': 'svc:/application/desktop-cache/docbook-dtds-update:default', 'objectName': 'api/com.oracle.solaris.rad.smf/1.0/Instance/application%2Fdesktop-cache%2Fdocbook-dtds-update,default', 'stime': 1601392118963, 'state': 'ONLINE'}, {'fmri': 'svc:/application/desktop-cache/docbook-style-dsssl-update:default', 'objectName': 'api/com.oracle.

It's not very easy to read and work with this response so we load it into a Pandas DataFrame which is much easier to read:

In [12]:
service_list_df = pd.DataFrame(service_list_text['payload'])
service_list_df.head()

,fmri,objectName,stime,state
0,svc:/application/cups/in-lpd:default,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,1601400264140,ONLINE
1,svc:/application/cups/scheduler:default,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,1601392118664,ONLINE
2,svc:/application/desktop-cache/desktop-mime-ca...,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,1601392633844,ONLINE
3,svc:/application/desktop-cache/docbook-dtds-up...,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,1601392118963,ONLINE
4,svc:/application/desktop-cache/docbook-style-d...,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,1601392621985,ONLINE


Now it's in a DataFrame we update the `stime` values to show the datetime value to make it more readable:

In [13]:
service_list_df['stime'] = pd.to_datetime(service_list_df['stime'], unit='ms')
service_list_df.head()

,fmri,objectName,stime,state
0,svc:/application/cups/in-lpd:default,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,2020-09-29 17:24:24.140,ONLINE
1,svc:/application/cups/scheduler:default,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,2020-09-29 15:08:38.664,ONLINE
2,svc:/application/desktop-cache/desktop-mime-ca...,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,2020-09-29 15:17:13.844,ONLINE
3,svc:/application/desktop-cache/docbook-dtds-up...,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,2020-09-29 15:08:38.963,ONLINE
4,svc:/application/desktop-cache/docbook-style-d...,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,2020-09-29 15:17:01.985,ONLINE


We've just shown the top five lines in the DataFrame with the `.head()` method to not make the output be too long, but maybe we want to search for a specific SMF service.<br>

For example the services named `rad:`, it's easy to pull them out like this:

In [14]:
service_list_df[service_list_df['fmri'].str.contains('rad:')]

,fmri,objectName,stime,state
272,svc:/system/rad:local,api/com.oracle.solaris.rad.smf/1.0/Instance/sy...,2020-10-02 08:48:22.529,ONLINE
273,svc:/system/rad:remote,api/com.oracle.solaris.rad.smf/1.0/Instance/sy...,2020-09-29 15:09:17.506,ONLINE


Note, they're both `ONLINE`. We're using `rad:remote` to connect to the server.

Similarly we can check the status of the FTP service:

In [15]:
service_list_df[service_list_df['fmri'].str.contains('ftp:')]

,fmri,objectName,stime,state
80,svc:/network/ftp:default,api/com.oracle.solaris.rad.smf/1.0/Instance/ne...,2020-09-29 15:14:11.198,DISABLED


Or which services are in the `DISABLED` state:

In [16]:
service_list_df[service_list_df['state'] == 'DISABLED']

,fmri,objectName,stime,state
17,svc:/application/management/net-snmp:default,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,2020-09-29 15:15:11.760,DISABLED
20,svc:/application/pkg/depot:default,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,2020-09-29 15:15:48.363,DISABLED
21,svc:/application/pkg/dynamic-mirror:default,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,2020-09-29 15:16:13.025,DISABLED
22,svc:/application/pkg/mirror:default,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,2020-09-29 15:16:28.179,DISABLED
25,svc:/application/pkg/sysrepo-cache-compact:def...,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,2020-09-29 15:19:46.772,DISABLED
...,...,...,...,...
280,svc:/system/sar:default,api/com.oracle.solaris.rad.smf/1.0/Instance/sy...,2020-09-29 15:07:30.889,DISABLED
287,svc:/system/svc/global:default,api/com.oracle.solaris.rad.smf/1.0/Instance/sy...,2020-09-29 15:08:29.613,DISABLED
295,svc:/system/system-log:rsyslog,api/com.oracle.solaris.rad.smf/1.0/Instance/sy...,2020-09-29 15:12:22.619,DISABLED
298,svc:/system/vtdaemon:default,api/com.oracle.solaris.rad.smf/1.0/Instance/sy...,2020-09-29 15:22:53.527,DISABLED


Counting how many services are in the each state:

In [17]:
service_list_df.groupby('state').count()['fmri']

state
DISABLED    120
ONLINE      185
Name: fmri, dtype: int64

**Note:** I'm using `['fmri']` at the end of this line to flatten the DataFrame to a Series as the data is the same in each column.

---
## Getting the Details of an Individual Service

Now we have the list of all the SMF services on the system we can get more details on a specfic service. 

Generally this would be used something like this:
`https://{{server}}:6788/api/com.oracle.solaris.rad.smf/1.0/Instance/<Service-Name>/<property>`

As you may have spotted this first part of the URI is already listed in the `objectName` column in the DataFrame above. 

So for the `rad:remote` instance we only have to search for it to get the full URI:

In [18]:
instance_uri = service_list_df[service_list_df['fmri'].str.contains('rad:remote')]['objectName'].iloc[0]
instance_uri

'api/com.oracle.solaris.rad.smf/1.0/Instance/system%2Frad,remote'

Now we can use this to get more information on this instance.

First we're going to get more detailed state information of this instance using the `/ex_state` property:

In [19]:
service_info = rad_rest_request('GET', s, server_connection_info, '{}/ex_state'.format(instance_uri))
service_info_text = json.loads(service_info.text)

print(service_info.status_code, '\n')
print(service_info_text)

200 

{'status': 'success', 'payload': {'state': 'ONLINE', 'nextState': 'NONE', 'auxstate': 'dependencies_satisfied', 'stime': 1601392157506, 'contractid': 172, 'enabled_state': True, 'enabled_temp': False}}


Printing it out so it's more readable:

In [20]:
for key, item in service_info_text['payload'].items():
    if key == 'stime':
        print(key, ':', timestamp_from_solaris_time(item))
    else:
        print(key, ':', item)

state : ONLINE
nextState : NONE
auxstate : dependencies_satisfied
stime : 2020-09-29 17:09:17.506000
contractid : 172
enabled_state : True
enabled_temp : False


We can now use this, taking the DataFrame we created above, add new columns for each of the service datafields and create a new larger table:

In [21]:
new_keys = set(service_list_df.columns.tolist() + list(service_info_text['payload'].keys()))

service_list_df = service_list_df.reindex(columns = new_keys)
service_list_df.head()

,enabled_temp,stime,fmri,auxstate,objectName,nextState,contractid,enabled_state,state
0,NaN,2020-09-29 17:24:24.140,svc:/application/cups/in-lpd:default,NaN,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,NaN,NaN,NaN,ONLINE
1,NaN,2020-09-29 15:08:38.664,svc:/application/cups/scheduler:default,NaN,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,NaN,NaN,NaN,ONLINE
2,NaN,2020-09-29 15:17:13.844,svc:/application/desktop-cache/desktop-mime-ca...,NaN,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,NaN,NaN,NaN,ONLINE
3,NaN,2020-09-29 15:08:38.963,svc:/application/desktop-cache/docbook-dtds-up...,NaN,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,NaN,NaN,NaN,ONLINE
4,NaN,2020-09-29 15:17:01.985,svc:/application/desktop-cache/docbook-style-d...,NaN,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,NaN,NaN,NaN,ONLINE


Now we have the larger table we can walk through each service in the list using its `objectName` and populate the new columns we've just created.

**Note:** Some services might throw an error, which you can catch by first checking if the call returns a `Message` key in its response.

**Note 2:** We're skipping `stime` because we already have it's value and this saves us converting it again. 

In [22]:
for loc, service_uri in enumerate(service_list_df['objectName']):
    service_info = rad_rest_request('GET', s, server_connection_info, '{}/ex_state'.format(service_uri))
    service_info_text = json.loads(service_info.text)
    
    if 'Message' in service_info_text['payload'].keys():
        print('Skipping {}'.format(service_uri))
    else:

        for key, item in service_info_text['payload'].items():
            if key == 'stime':
                pass
            else:
                service_list_df[key].iloc[loc] = item

service_list_df.head()

Skipping api/com.oracle.solaris.rad.smf/1.0/Instance/system%2Fearly-manifest-import,default


,enabled_temp,stime,fmri,auxstate,objectName,nextState,contractid,enabled_state,state
0,False,2020-09-29 17:24:24.140,svc:/application/cups/in-lpd:default,none,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,NONE,-1.0,True,ONLINE
1,False,2020-09-29 15:08:38.664,svc:/application/cups/scheduler:default,dependencies_satisfied,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,NONE,96.0,True,ONLINE
2,False,2020-09-29 15:17:13.844,svc:/application/desktop-cache/desktop-mime-ca...,dependencies_satisfied,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,NONE,-1.0,True,ONLINE
3,False,2020-09-29 15:08:38.963,svc:/application/desktop-cache/docbook-dtds-up...,dependencies_satisfied,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,NONE,-1.0,True,ONLINE
4,False,2020-09-29 15:17:01.985,svc:/application/desktop-cache/docbook-style-d...,dependencies_satisfied,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,NONE,-1.0,True,ONLINE


Again having this all in a Pandas DataFrame makes it easy to find something complex like for example the `objectName` of the last currently `ONLINE` service to come online:

In [23]:
service_list_df[service_list_df['state'] == 'ONLINE'].sort_values(by=['stime']).iloc[-1].objectName

'api/com.oracle.solaris.rad.smf/1.0/Instance/system%2Fwebui%2Fserver,default'

---
### Getting the Property Groups and Values

Now to switch to looking at the **Property Groups** and **Property Values** of a specific service. The values you get when running `svccfg -s <service-name> listprop`.

The service we want to look at is `compliance:default`, the service that allows you schedule Compliance assessments to run at a regular cadence. 

First looking up the service in the DataFrame:

In [24]:
service_list_df[service_list_df['fmri'].str.contains('compliance:default')]

,enabled_temp,stime,fmri,auxstate,objectName,nextState,contractid,enabled_state,state
28,False,2020-09-29 15:19:48.457,svc:/application/security/compliance:default,none,api/com.oracle.solaris.rad.smf/1.0/Instance/ap...,NONE,-1.0,False,DISABLED


It's disabled. 

Now to grab the URI:

In [25]:
instance_uri = service_list_df[service_list_df['fmri'].str.contains('compliance:default')]['objectName'].iloc[0]
instance_uri

'api/com.oracle.solaris.rad.smf/1.0/Instance/application%2Fsecurity%2Fcompliance,default'

Now we can go an load the **Property Groups**:

In [26]:
service_pgs = rad_rest_request('GET', s, server_connection_info, '{}/pgs'.format(instance_uri))
service_pgs_text = json.loads(service_pgs.text)

print(service_pgs.status_code, '\n')
print(service_pgs_text)

pgs_list_df = pd.DataFrame(service_pgs_text['payload'])
pgs_list_df

200 

{'status': 'success', 'payload': [{'name': 'general', 'type': 'framework', 'flags': 0}, {'name': 'options', 'type': 'application', 'flags': 0}, {'name': 'periodic_restarter', 'type': 'framework', 'flags': 0}, {'name': 'policy', 'type': 'application', 'flags': 0}, {'name': 'restarter', 'type': 'framework', 'flags': 1}, {'name': 'scheduled', 'type': 'schedule', 'flags': 0}, {'name': 'start', 'type': 'method', 'flags': 0}, {'name': 'tm_common_name', 'type': 'template', 'flags': 0}, {'name': 'tm_man_compliance8', 'type': 'template', 'flags': 0}]}


,name,type,flags
0,general,framework,0
1,options,application,0
2,periodic_restarter,framework,0
3,policy,application,0
4,restarter,framework,1
5,scheduled,schedule,0
6,start,method,0
7,tm_common_name,template,0
8,tm_man_compliance8,template,0


This gives the list of Property Groups, but we're more interested in the Property Values in each group.

---
### Reading the Property Group Values

To retrieve the values of an individual Property Group we can use a **PUT** method and add the name of this group into a list labeled `'pg_names'` in the JSON text we send as part of the **PUT**.

For example if we're interested in the values in the `scheduled` Property Group:

In [27]:
json_body = {'pg_names': ['scheduled']}

In [28]:
service_pgval = rad_rest_request('PUT', s, server_connection_info, '{}/_rad_method/readProperties'.format(instance_uri), json_body)
service_pgval_text = json.loads(service_pgval.text)

print(service_pgval.status_code, '\n')
print(service_pgval_text)

pgval_list_df = pd.DataFrame(service_pgval_text['payload'])
pgval_list_df

200 

{'status': 'success', 'payload': [{'name': 'scheduled/frequency', 'type': 'INTEGER', 'values': ['1']}, {'name': 'scheduled/interval', 'type': 'ASTRING', 'values': ['week']}, {'name': 'scheduled/modify_authorization', 'type': 'ASTRING', 'values': ['solaris.compliance.assess']}]}


,name,type,values
0,scheduled/frequency,INTEGER,[1]
1,scheduled/interval,ASTRING,[week]
2,scheduled/modify_authorization,ASTRING,[solaris.compliance.assess]


Similarly we can get the Property Values for all the Propery Groups currently set by creating a list of all the Property Groups and sending this as part of the JSON text.

So first we extract the Property Group names from the DataFrame and put them into JSON:

In [29]:
json_body = {"pg_names": pgs_list_df['name'].to_list()}
json_body

{'pg_names': ['general',
  'options',
  'periodic_restarter',
  'policy',
  'restarter',
  'scheduled',
  'start',
  'tm_common_name',
  'tm_man_compliance8']}

Then we run the same **PUT** method and get the full list back:

In [30]:
service_pgval = rad_rest_request('PUT', s, server_connection_info, '{}/_rad_method/readProperties'.format(instance_uri), json_body)
service_pgval_text = json.loads(service_pgval.text)

print(service_pgval.status_code, '\n')
print(service_pgval_text)

pgval_list_df = pd.DataFrame(service_pgval_text['payload'])
pgval_list_df

200 

{'status': 'success', 'payload': [{'name': 'general/action_authorization', 'type': 'ASTRING', 'values': ['solaris.smf.manage.compliance']}, {'name': 'general/complete', 'type': 'ASTRING', 'values': []}, {'name': 'general/enabled', 'type': 'BOOLEAN', 'values': ['false']}, {'name': 'general/restarter', 'type': 'FMRI', 'values': ['svc:/system/svc/periodic-restarter:default']}, {'name': 'general/value_authorization', 'type': 'ASTRING', 'values': ['solaris.compliance.assess']}, {'name': 'options/assessment', 'type': 'ASTRING', 'values': ['']}, {'name': 'options/matches', 'type': 'ASTRING', 'values': ['']}, {'name': 'options/store-URI', 'type': 'ASTRING', 'values': ['']}, {'name': 'options/value_authorization', 'type': 'ASTRING', 'values': ['solaris.compliance.assess']}, {'name': 'periodic_restarter/next_run', 'type': 'TIME', 'values': ['1600941026']}, {'name': 'periodic_restarter/scheduled', 'type': 'TIME', 'values': ['0']}, {'name': 'policy/benchmark', 'type': 'ASTRING', 'values': ['

,name,type,values
0,general/action_authorization,ASTRING,[solaris.smf.manage.compliance]
1,general/complete,ASTRING,[]
2,general/enabled,BOOLEAN,[false]
3,general/restarter,FMRI,[svc:/system/svc/periodic-restarter:default]
4,general/value_authorization,ASTRING,[solaris.compliance.assess]
5,options/assessment,ASTRING,[]
6,options/matches,ASTRING,[]
7,options/store-URI,ASTRING,[]
8,options/value_authorization,ASTRING,[solaris.compliance.assess]
9,periodic_restarter/next_run,TIME,[1600941026]


This is essentially the same as running `svccfg -s compliance:default listprop` on the CLI.

---
### Setting SMF Properties

In our case we're not only interested in looking at the Property Values of an SMF service, but also at how to set them to a different value.

To achieve this we use the `/_rad_method/writeProperties` property which calls the RAD method `writeProperties` on the Instance interface using a **PUT** method. Yes, there's a lot of overlapping use of the words *method* and *property* between REST, RAD, and SMF.

In any even the URI looks like this:

`api/com.oracle.solaris.rad.smf/1.0/Instance/<instance-name>/_rad_method/writeProperties`

We'll also need to send the name of the Property Value and the value you want to set it to in a JSON text. So for example if for the `compliance:default` service instance we want to set `scheduled/interval` to `hour` instead of the current value of `week` we'd create a JSON text like this:

```
{
    "props": [
        {
            "name": "scheduled/interval",
            "type": "ASTRING",
            "values": [
                "hour"
            ]
        }
    ]
}
```

And then send it with the URI we've been using up to now:

In [31]:
json_body = {'props': [{'name': 'scheduled/interval', 'type': 'ASTRING', 'values': ['hour']}]}

In [32]:
service_setval = rad_rest_request('PUT', s, server_connection_info, '{}/_rad_method/writeProperties'.format(instance_uri), json_body)
service_setval_text = json.loads(service_setval.text)

print(service_setval.status_code, '\n')
print(service_setval_text)

200 

{'status': 'success', 'payload': None}


This should return the `status` as `success`, which means the value is set.

Let quickly check:

In [33]:
json_body = {"pg_names": ["scheduled"]}

In [34]:
service_pgval = rad_rest_request('PUT', s, server_connection_info, '{}/_rad_method/readProperties'.format(instance_uri), json_body)
service_pgval_text = json.loads(service_pgval.text)

print(service_pgval_text['payload'][1]['values'][0])

hour


It should show `hour`.

---
### Enabling SMF service

Now the value has been changed we may want to turn the `compliance:default` service instance on. We do this by using the `_rad_method/enable` property with the instance URI of the service instance with a **PUT** method. 

Additionally we'll need to indicate if this service is only temporarily turned on or not with the follwoing JSON text:

```
{
    "temporary": false
}
```

Setting the text and turning the service instance on:

In [35]:
json_body = {'temporary': False}

In [36]:
service_enable = rad_rest_request('PUT', s, server_connection_info, '{}/_rad_method/enable'.format(instance_uri), json_body)
service_enable_text = json.loads(service_enable.text)

print(service_enable.status_code, '\n')
print(service_enable_text)

200 

{'status': 'success', 'payload': None}


Again here the `status` should return as `success`.

We can quickly check this:

In [37]:
service_info = rad_rest_request('GET', s, server_connection_info, '{}/ex_state'.format(instance_uri))
service_info_text = json.loads(service_info.text)

print(service_info.status_code, '\n')
print(service_info_text['payload']['state'])

200 

ONLINE


This should return `ONLINE`.

---
### Disabling SMF service

To finish this notebook we'll show how to turn the service off again. This is essentially the same as starting the service but this time using the `_rad_method/disable` property. The rest is all the same:

In [38]:
json_body = {'temporary': False}

In [39]:
service_enable = rad_rest_request('PUT', s, server_connection_info, '{}/_rad_method/disable'.format(instance_uri), json_body)
service_enable_text = json.loads(service_enable.text)

print(service_enable.status_code, '\n')
print(service_enable_text)

200 

{'status': 'success', 'payload': None}


Again look for `success`.

And checking it:

In [40]:
service_info = rad_rest_request('GET', s, server_connection_info, '{}/ex_state'.format(instance_uri))
service_info_text = json.loads(service_info.text)

print(service_info.status_code, '\n')
print(service_info_text['payload']['state'])

200 

DISABLED


`DISABLED` should be what is returned.

This brings us to the end of the notebook. 

---
### Putting the Value Back

To finish it off completely you can also put the value we changed back to `week`:

In [41]:
json_body = {'props': [{'name': 'scheduled/interval', 'type': 'ASTRING', 'values': ['week']}]}

In [42]:
service_setval = rad_rest_request('PUT', s, server_connection_info, '{}/_rad_method/writeProperties'.format(instance_uri), json_body)
service_setval_text = json.loads(service_setval.text)

print(service_setval.status_code, '\n')
print(service_setval_text)

200 

{'status': 'success', 'payload': None}


---
---

### SMF URIs Used

List of SMF services <br>
`https://{{server}}:6788/api/com.oracle.solaris.rad.smf/1.0/Master/instances`
<br><br>
Get the state of a specific instance<br>
`https://{{server}}:6788/api/com.oracle.solaris.rad.smf/1.0/Instance/network%2Fip-interface-management,default/ex_state`
<br><br>
Get the property groups of an instance<br>
`https://{{server}}:6788/api/com.oracle.solaris.rad.smf/1.0/Instance/application%2Fsecurity%2Fcompliance,default/pgs`
<br><br>
Read the values of the property groups<br>
`https://{{server}}:6788/api/com.oracle.solaris.rad.smf/1.0/Instance/application%2Fsecurity%2Fcompliance,default/_rad_method/readProperties`<br>
with this body text
```
{
    "pg_names": [
        "scheduled"
    ]
}
```
<br><br>
Set SMF properties<br>
`https://{{server}}:6788/api/com.oracle.solaris.rad.smf/1.0/Instance/application%2Fsecurity%2Fcompliance,default/_rad_method/writeProperties`<br>
with this body text
```
{
    "props": [
        {
            "name": "scheduled/interval",
            "type": "ASTRING",
            "values": [
                "hour"
            ]
        }
    ]
}
```
<br><br>
Enable SMF service<br>
`https://{{server}}:6788/api/com.oracle.solaris.rad.smf/1.0/Instance/application%2Fsecurity%2Fcompliance,default/_rad_method/enable`<br>
with this body text
```
{
    "temporary": false
}
```
<br><br>
Disable SMF service<br>
`https://{{server}}:6788/api/com.oracle.solaris.rad.smf/1.0/Instance/application%2Fsecurity%2Fcompliance,default/_rad_method/disable`<br>
with this body text
```
{
    "temporary": false
}
```